## Data Scraping

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [25]:
from bs4 import BeautifulSoup
from pattern.web import download
import pandas as pd

In [92]:
html_doc = download('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(html_doc,'lxml')
wiki_table = soup.find('table',class_='wikitable').find_all('tr')
toronto_data = []
for index, row in enumerate(wiki_table):
    if index == 0:
        pass
    else:
        data = row.find_all('td')
        postcode = data[0].text
        borough = data[1].text
        neighborhood = data[2].text.strip()
        toronto_data.append([postcode,borough,neighborhood])
toronto_df = pd.DataFrame(toronto_data, columns=['PostCode','Borough','Neighborhood'])
toronto_df.head()

,PostCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [93]:
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.head()

,PostCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


More than one neighborhood can exist in one postal code area.

In [94]:
toronto_df['Neighborhood'] = toronto_df.groupby(['PostCode','Borough']).transform(lambda x: ', '.join(x))
toronto_df = toronto_df.drop_duplicates()
toronto_df.head()

,PostCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
7,M7A,Downtown Toronto,Queen's Park


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [95]:
toronto_df['Neighborhood'] = toronto_df.apply(lambda row: row['Borough'] if row['Neighborhood'] == 'Not assigned' else row['Neighborhood'], axis=1)

In [98]:
toronto_df.shape

(103, 3)